# Goose Test Page
This is the test page that is supposed to analyse the goose model data.<br>
First let us define the location of the data to analyse.

In [5]:
data_dir = "~/CLionProjects/GooseTests/run-directory/"
source_dir = "~/CLionProjects/ALMaSS_all"